In [113]:
import cohere
import os
import json
from pprint import pprint

from dotenv import load_dotenv
import os

load_dotenv()

True

In [114]:
co = cohere.Client(os.getenv('COHERE_API_KEY'))

examples = []
inputs = []

with open('../documents/labels.json') as f:
    data = json.load(f)
    for file in os.listdir('../documents'):
        if file.endswith('.md') and file in data:
            with open('../documents/' + file) as example:
                examples.append(cohere.responses.classify.Example(example.read(), data[file]))

file_list = []
input_list = []

for file in os.listdir('../documents'):
    if file.endswith('.md'):
        with open('../documents/' + file) as example:
            inputs.append({"name": file, "data": example.read()})

print(len(inputs), len(examples))

response = co.classify(
    model='large',
    inputs=list(map(lambda x: x['data'], inputs)),
    examples=examples
)
doc_map = {}
for idx, document in enumerate(inputs):
    print('Name', document['name'])
    doc_map[document['name']] = response.classifications[idx].prediction
    print('Classification', response.classifications[idx].prediction)
    print('')



18 15
Name palantir.md
Classification proposal

Name RestAPI.md
Classification proposal

Name quarterly.md
Classification memo

Name test_incomplete.md
Classification memo

Name weekly_memo.md
Classification memo

Name JavaToKotlin.md
Classification report

Name AWSGuide.md
Classification guide

Name website_guide.md
Classification guide

Name dummy_proposal.md
Classification proposal

Name snacks.md
Classification memo

Name stretch.md
Classification guide

Name test_proposal.md
Classification report

Name ActionsToJenkins.md
Classification report

Name java.md
Classification report

Name AIGuide.md
Classification guide

Name InhouseToAWS.md
Classification report

Name Remote.md
Classification memo

Name ai_proposal.md
Classification proposal



In [115]:
with open('data.json', 'r+') as f:
    data = json.load(f)
    for item in data:
        if item['meta']['source'] in doc_map:
            item['meta']['classification'] = doc_map[item['meta']['source']]

    f.seek(0)
    json.dump(data, f)
    f.truncate()
